## Project Euler Solution #54

In [15]:
from itertools import combinations

*** Think about mapping each hand to a unique score... how could that be done? ***

High Card: Highest value card.

One Pair: Two cards of the same value.

Two Pairs: Two different pairs.

Three of a Kind: Three cards of the same value.

Straight: All cards are consecutive values.

Flush: All cards of the same suit.

Full House: Three of a kind and a pair.

Four of a Kind: Four cards of the same value.

Straight Flush: All cards are consecutive values of same suit.

Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.



In [55]:
# functions
card_ordering = [str(i) for i in range(2, 10)] + ['T', 'J', 'Q', 'K', 'A']
card_priority = {char: i for i, char in enumerate(card_ordering[::-1])}
card_vals = {'2': 2, '3': 3, '4': 4,  '5': 5, '6': 6, '7': 7,  '8': 8,  '9': 9, 'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}

def sort_key(item):
    if len(item) != 2:
        raise ValueError("Must be a 2-character card string.")
    return card_priority[item[0]]

def sort_hand(hand):
    """Takes in a hand and returns the sorted hand of cards and the sorted list of suits."""
    sorted_hand = sorted(hand, key=sort_key)

    cards = [card_vals[card[0]] for card in sorted_hand]
    suits = [card[1] for card in sorted_hand]
    return cards, suits

def hand_score(cards, suits):
    """Returns a unique score for the hand."""
    straight = False
    flush = False
    three_o_kind = False
    four_o_kind = False
    full_house = False
    two_pair = False
    one_pair = False
    nothing = False
    highest = cards[0]

    if all([s == suits[0] for s in suits[1:]]):
        flush = True

    if highest - cards[-1] == 4:
        straight = True

    quart_sum = 0
    for quart in combinations(cards, 4):
        if all([val == quart[0] for val in quart[1:]]):
            four_o_kind = True
            quart_sum += quart[0] * 3

    trip_sum = 0
    for trip in combinations(cards, 3):
        if all([val == trip[0] for val in trip[1:]]):
            three_o_kind = True
            trip_sum += trip[0] * 3

    if not four_o_kind:        
        num_pairs = 0
        pair_nums = [0]
        trip_num = trip_sum // 3
        for pair in combinations(cards, 2):
            if pair[0] == pair[1] and pair[0] != trip_num:
                num_pairs += 1
                pair_nums.append(pair[0])
                highest = [val for val in cards if val != pair[0]][0]
             
        if num_pairs == 1:
            one_pair = True
        if num_pairs == 2:
            two_pair = True
    
    if not straight and not flush and not three_o_kind and not four_o_kind and not (two_pair or one_pair):
        nothing = True

    full_house = three_o_kind and one_pair
    straight_flush = flush and straight

    score = straight_flush * (highest + 264 + (238*4)) + four_o_kind * (quart_sum + 208 + (238*4)) + full_house * (trip_sum + 166 + (238*4)) + flush * (highest + 152 + (238*4)) + straight * (highest + 138 + (238*4)) + three_o_kind * (trip_sum + 96 + (238*4)) + two_pair * (highest + sum(pair_nums) * 16 + 42 + 238) + one_pair * (highest + max(pair_nums) * 16 + 14) + nothing * highest
    # + (238*4)
    return score

def winner(p1, p2):
    """Takes two power hands, p1 and p2, and returns the winner (1 or 2)."""
    # sort the hands using our cool functions
    p1_cards, p1_suits = sort_hand(p1)
    p2_cards, p2_suits = sort_hand(p2)

    # score the hand
    p1_score = hand_score(p1_cards, p1_suits)
    p2_score = hand_score(p2_cards, p2_suits)
    print("p1 score: ", p1_score, "\tp2 score: ", p2_score)
    # return the winner (after a final check)
    if p1_score == p2_score:
        raise ValueError(f"Your scoring is broken with hands {p1} and {p2}.")
    return 1 if p1_score > p2_score else 2

In [56]:
# tests!
with open("./data/poker.txt") as f:
    all_hands = [line.strip().split(' ') for line in f.readlines()]

for hand in all_hands[:2]:
    p1 = hand[:5]
    p2 = hand[5:]
    print(p1, p2)
    print(sort_hand(p1), sort_hand(p2))
    print(f"The winner is Player {winner(p1, p2)}!\n")

['8C', 'TS', 'KC', '9H', '4S'] ['7D', '2S', '5D', '3S', 'AC']
([13, 10, 9, 8, 4], ['C', 'S', 'H', 'C', 'S']) ([14, 7, 5, 3, 2], ['C', 'D', 'D', 'S', 'S'])
p1 score:  13 	p2 score:  14
The winner is Player 2!

['5C', 'AD', '5D', 'AC', '9C'] ['7C', '5H', '8D', 'TD', 'KS']
([14, 14, 9, 5, 5], ['D', 'C', 'C', 'C', 'D']) ([13, 10, 8, 7, 5], ['S', 'D', 'D', 'C', 'H'])
p1 score:  598 	p2 score:  13
The winner is Player 1!



In [57]:
# read in the data
with open("./data/poker.txt") as f:
    all_hands = [line.strip().split(' ') for line in f.readlines()]

count = 0
for hand in all_hands:
    p1 = hand[:5]
    p2 = hand[5:]
    if winner(p1, p2) == 1:
        count += 1

print(f"Player 1 won {count} hands.")

p1 score:  13 	p2 score:  14
p1 score:  598 	p2 score:  13
p1 score:  13 	p2 score:  12
p1 score:  186 	p2 score:  203
p1 score:  13 	p2 score:  582
p1 score:  171 	p2 score:  108
p1 score:  1096 	p2 score:  14
p1 score:  182 	p2 score:  229
p1 score:  13 	p2 score:  220
p1 score:  57 	p2 score:  14
p1 score:  14 	p2 score:  248
p1 score:  104 	p2 score:  13
p1 score:  14 	p2 score:  13
p1 score:  13 	p2 score:  123
p1 score:  8 	p2 score:  90
p1 score:  13 	p2 score:  14
p1 score:  123 	p2 score:  250
p1 score:  1610 	p2 score:  12
p1 score:  152 	p2 score:  1075
p1 score:  12 	p2 score:  10
p1 score:  12 	p2 score:  140
p1 score:  14 	p2 score:  13
p1 score:  14 	p2 score:  76
p1 score:  11 	p2 score:  14
p1 score:  1198 	p2 score:  105
p1 score:  13 	p2 score:  198
p1 score:  14 	p2 score:  13
p1 score:  12 	p2 score:  14
p1 score:  107 	p2 score:  13
p1 score:  164 	p2 score:  11
p1 score:  14 	p2 score:  12
p1 score:  14 	p2 score:  10
p1 score:  172 	p2 score:  11
p1 score:  13 	